This notebooks runs model experiments using the Catboost and TabPFN model architectures which benefit from GPU computation.

In [9]:
import os
import psutil
import sys
from pathlib import Path

In [10]:
# Check if running in Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
		# Install required packages
    !pip install -q pandas numpy matplotlib seaborn sklearn tabpfn shap

		# Add tabpfn extensions
    !git clone https://github.com/PriorLabs/tabpfn-extensions
    !pip install -e tabpfn-extensions[all]
		
    # Download required files from GCS
    project = 'fwi-water-quality-sensing'
    bucket_name = 'fwi-predict'
    local_predict_df_path = '/tmp/predict_df.csv'
    local_sentinel_path = '/tmp/sentinel2.csv'
    
    !gcloud config set project {project}
    !gsutil cp gs://{bucket_name}/train/predict_dfs/jun_21_dec_24_w_metadata_predict_df.csv {local_predict_df_path}
    !gsutil cp gs://{bucket_name}/train/sentinel2/jun_21_dec_24_w_metadata.csv {local_sentinel_path}
else:
    data_dir = Path('../../data')
    local_predict_df_path = data_dir / "predict_dfs/train/jun_21_dec_24_w_metadata_predict_df.csv"
    local_sentinel_path = data_dir / "gcs/train/sentinel2/jun_21_dec_24_w_metadata.csv"


In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [4]:
# Load and merge
predict_df = pd.read_csv(local_predict_df_path, parse_dates=['sample_dt'], index_col=0)
sentinel_features = pd.read_csv(local_sentinel_path).drop(columns=['system:index', '.geo'])
predict_df = predict_df.merge(sentinel_features, on='sample_idx')

# Drop unhelpful features
num_sum_cols = predict_df.columns[predict_df.columns.str.contains('num_sum')].tolist()
drop_cols = ['sample_idx', 'geometry', 'sample_dt', 'prev_sample_dt'] + num_sum_cols
predict_df = predict_df.drop(columns=drop_cols)

# Get parameters for classification problem
# predict_df['month'] = predict_df['sample_dt'].dt.month # Not including for now since last year had odd trend I don't want attributed to month

predict_df = predict_df.drop(columns=['ammonia_mg_per_L', 'ph', 'turbidity_cm'])
winkler_df = predict_df[predict_df['do_winkler']].copy()

In [5]:
categoricals = ['farm', 'region', 'month', 'pond_id', 'village', 'SCL']
non_categoricals = winkler_df.columns[~winkler_df.columns.isin(categoricals)].tolist()

In [6]:
experiment_df = winkler_df.copy()

In [7]:
# Feature categories
base_weather_forecast_features = experiment_df.columns[experiment_df.columns.str.contains('downward_shortwave|precipitable|relative_hum|specific_hum|temperature_2m|total_cloud|total_precip')].tolist()
ext_weather_forecast_features = base_weather_forecast_features + experiment_df.columns[experiment_df.columns.str.contains('u_component|v_component')].tolist()
satellite_features = ['AOT', 'B1', 'B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'MSK_CLDPRB', 'SCL', 'WVP', 'hours_from_measurement', 'ndvi','ndwi']
lag_features = experiment_df.columns[experiment_df.columns.str.startswith('prev_')].tolist()
pond_features = (
  experiment_df.columns[experiment_df.columns.str.startswith('fertilizer_|feed_|has_')].tolist() + ['pond_area_acres', 'property_area_acres', 'pond_depth_meters', 'pond_preparation']
)

# Feature sets
base_feature_set = ['morning', 'region', 'hour'] + base_weather_forecast_features
ext_weather_forecast_set = base_feature_set + ext_weather_forecast_features

# Add categoricals
base_w_categoricals = list(set(base_feature_set + categoricals))
base_w_lag_features = base_feature_set + lag_features
base_w_satellite_features = base_feature_set + satellite_features
base_w_pond_features = base_feature_set + pond_features

In [8]:
if IN_COLAB:
		# Make sure reflects latest versions in local repository
		WQ_RANGES = {
				'do_mg_per_L': {
						'required': {
								'morning': (3, 5),
								'evening': (8, 12)
						},
						'ideal': {
								'morning': (4, 5),
								'evening': (8, 10)
						}
				},
				'ph': {
						'required': (6.5, 8.5),
						'ideal': (7, 8)
				},
				'ammonia_mg_per_L': {
						'required': (0, 0.5),
						'ideal': (0, 0.15)
				},
				'turbidity_cm': {
						'required': (20, 50),
						'ideal': (30, 40)
				}
		}
  
		def get_in_required_range(parameter: str, values, periods=None):
				"""Checks if water quality parameter is below, within, or above the required range.

				Parameters:
						parameter (str): The water quality parameter to check (e.g., 'do', 'ph', 'ammonia', 'turbidity').
						values: Array-like of measurement values (numpy array or pandas Series).
						periods: Array-like of periods ('morning', 'evening', etc.), required for period-dependent parameters.

				Returns:
						Array-like: Array of strings indicating if values are 'below', 'within', or 'above' the required range.
				"""
				# Convert inputs to numpy arrays for consistent handling
				values = np.asarray(values)
				
				# Ensure the parameter is valid
				if parameter not in WQ_RANGES:
						raise ValueError(f"Invalid parameter: {parameter}. Must be one of {list(WQ_RANGES.keys())}.")
				
				required_ranges = WQ_RANGES[parameter]['required']
				
				# Handle case where ranges are split by periods
				if isinstance(required_ranges, dict):
						if periods is None:
								raise ValueError(f"Periods must be provided for parameter {parameter}")
								
						periods = np.asarray(periods)
						result = np.full(values.shape, '', dtype='U6') # Initialize output array

						# Process each period
						for period, (low, high) in required_ranges.items():
								mask = periods == period
								conditions = [values[mask] < low, (values[mask] >= low) & (values[mask] <= high), values[mask] > high]
								result[mask] = np.select(conditions, ['below', 'within', 'above'], default='')
								
						return result
				
				# Handle case where ranges are not split by periods
				low, high = required_ranges
				conditions = [values < low, (values >= low) & (values <= high), values > high]
				return np.select(conditions, ['below', 'within', 'above'], default='')
		
else:
		from fwi_predict.constants import WQ_RANGES
		from fwi_predict.wq import get_in_required_range

In [12]:
# Import necessary libraries
from typing import List, Dict, Tuple

import shap
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
from sklearn.metrics import root_mean_squared_error, precision_score, recall_score, r2_score
from sklearn.model_selection import cross_val_predict, GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, TargetEncoder
from tabpfn import TabPFNRegressor
from tabpfn_extensions.post_hoc_ensembles.sklearn_interface import AutoTabPFNRegressor
from tabpfn_extensions.interpretability.shap import get_shap_values, plot_shap

In [ ]:
# Define diurnal detrend transform
class DiurnalDetrend(BaseEstimator, TransformerMixin):
    """Detrend data by subtracting morning/evening means."""
    def fit(self, X, y=None):
        # Calculate means for morning/evening
        df = pd.DataFrame({'y': y, 'morning': X['morning']})
        self.morning_mean_ = df[df['morning']]['y'].mean()
        self.evening_mean_ = df[~df['morning']]['y'].mean()
        return self

    def transform(self, X, y=None):
        if y is not None:
            y_detrended = y.copy()
            y_detrended[X['morning']] -= self.morning_mean_
            y_detrended[~X['morning']] -= self.evening_mean_
            return y_detrended
        return X
    
    def inverse_transform(self, X, y):
        y_retrended = y.copy()
        y_retrended[X['morning']] += self.morning_mean_
        y_retrended[~X['morning']] += self.evening_mean_
        return y_retrended
    
    def get_feature_names_out(self, input_features=None):
        """Get output feature names for transformation.
        
        Parameters
        ----------
        input_features : list of str or None, default=None
            Input feature names. If None, then feature_names_in_ is used.
            
        Returns
        -------
        feature_names_out : ndarray of str
            Output feature names.
        """
        if input_features is None:
            input_features = self.feature_names_in_
        return np.asarray(input_features)

In [ ]:
workers = psutil.cpu_count() if IN_COLAB else psutil.cpu_count(logical=False) - 1
print(f"Workers: {workers}")

Workers: 9


In [ ]:
def evaluate_predictions(y_true, y_pred, morning_series):
    """Evaluate model performance including DO range classification."""
    # Get range classifications
    time_of_day = morning_series.apply(lambda x: 'morning' if x else 'evening')
    true_ranges = get_in_required_range('do_mg_per_L', y_true, time_of_day)
    pred_ranges = get_in_required_range('do_mg_per_L', y_pred, time_of_day)
    
    # Calculate metrics
    r2 = r2_score(y_true, y_pred)
    range_accuracy = (true_ranges == pred_ranges).mean()
    
    return {
        'r2_score': r2,
        'range_accuracy': range_accuracy,
        'y_true': y_true,
        'y_pred': y_pred,
        'true_ranges': true_ranges,
        'pred_ranges': pred_ranges
    }



def run_experiment(df: pd.DataFrame, 
                  feature_set: List[str],
                  categoricals: List[str],
                  feature_selection: bool = False,
                  diurnal_detrend: bool = True,
                  scoring: str = 'neg_root_mean_squared_error',
                  workers: int = workers) -> Dict:
    """Run modeling experiment with given feature set."""
    
    # Prepare data
    X = df[feature_set]
    y = df['do_mg_per_L']
    
		# Main training pipeline without feature selection
    categorical_features = [col for col in feature_set if col in categoricals]
    categorical_feature_indices = [feature_set.index(col) for col in categorical_features]
    pipeline = Pipeline([
        ('detrend', DiurnalDetrend() if diurnal_detrend else 'passthrough'),
        ('regressor', TabPFNRegressor(
            categorical_features_indices=categorical_feature_indices,
            n_jobs=workers,
            random_state=50
        ))
    ])
    
    # Define parameter grid
    param_grid = {
        'regressor__n_estimators': [4, 8, 14]
		}
    
    # Perform grid search
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=cv,
        scoring=scoring,
        n_jobs=workers,
        verbose=1
    )
    
    # Fit model
    grid_search.fit(X, y)
    
    # Get cross-validation predictions
    best_estimator = grid_search.best_estimator_
    cv_predictions = cross_val_predict(
        best_estimator,
        X, y,
        cv=cv
    )
    
    # Evaluate performance
    results = evaluate_predictions(y, cv_predictions, df['morning'])
    
    # Plot hyperparameter effects
    cv_results = pd.DataFrame(grid_search.cv_results_)
    # Get hyperparameter columns
    param_cols = [col for col in cv_results.columns if col.startswith('param_regressor__')]
    n_params = len(param_cols)
    
    # Calculate subplot layout
    n_rows = int(np.ceil(np.sqrt(n_params)))
    n_cols = int(np.ceil(n_params / n_rows))
    
    # Create figure and axes
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(7*n_cols, 5*n_rows))
    axes = axes.flatten()
    
    # Create plots for each hyperparameter
    for i, param_col in enumerate(param_cols):
        param_name = param_col.replace('param_regressor__', '')
        sns.boxplot(data=cv_results, x=param_col, y='mean_test_score', ax=axes[i])
        axes[i].set_title(f'Effect of {param_name}')
        axes[i].set_ylabel('Mean CV Score (R²)')
        
    # Hide any empty subplots
    for i in range(n_params, len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()
    
    # Plot cross-validation fold performance
    cv_fold_scores = []
    for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X)):
        y_true_fold = y.iloc[test_idx]
        y_pred_fold = cv_predictions[test_idx]
        
        # Calculate classification metrics using WQ ranges
        true_classes = get_in_required_range('do_mg_per_L', y_true_fold, ['morning']*len(y_true_fold))
        pred_classes = get_in_required_range('do_mg_per_L', y_pred_fold, ['morning']*len(y_pred_fold))
        
        # Calculate per-class metrics
        for class_name in ['below', 'within', 'above']:
            precision = precision_score(true_classes == class_name, pred_classes == class_name)
            recall = recall_score(true_classes == class_name, pred_classes == class_name)
            
            cv_fold_scores.append({
                'fold': fold_idx + 1,
                'class': class_name,
                'precision': precision,
                'recall': recall,
            })
    
    cv_scores_df = pd.DataFrame(cv_fold_scores)
    
    # Plot fold performance comparisons
    plt.figure(figsize=(10, 5))
    plot_data = cv_scores_df.melt(id_vars=['fold', 'class'], 
                                 value_vars=['precision', 'recall'],
                                 var_name='metric', value_name='score')
    
    sns.boxplot(data=plot_data, x='class', y='score', hue='metric')
    plt.title('Precision and recall scores by class across folds')
    plt.tight_layout()
    plt.show()
    
    # Plot predictions vs actuals
    plt.figure(figsize=(15, 5))
    
    # Morning predictions
    plt.subplot(1, 2, 1)
    morning_mask = df['morning']
    plt.scatter(results['y_true'][morning_mask], 
                results['y_pred'][morning_mask], alpha=0.5)
    plt.plot([results['y_true'][morning_mask].min(), 
              results['y_true'][morning_mask].max()],
             [results['y_true'][morning_mask].min(), 
              results['y_true'][morning_mask].max()], 
             'r--', lw=2, label='Perfect prediction')
    
    # Add trend line
    z = np.polyfit(results['y_true'][morning_mask], results['y_pred'][morning_mask], 1)
    p = np.poly1d(z)
    plt.plot([results['y_true'][morning_mask].min(), 
              results['y_true'][morning_mask].max()],
             [p(results['y_true'][morning_mask].min()), 
              p(results['y_true'][morning_mask].max())],
             'g--', lw=2, label='Trend line')
    plt.legend()
    plt.xlabel('Actual DO (mg/L)')
    plt.ylabel('Predicted DO (mg/L)')
    plt.title('Morning: Predicted vs actual DO levels')
    
    # Evening predictions
    plt.subplot(1, 2, 2)
    evening_mask = ~df['morning']
    plt.scatter(results['y_true'][evening_mask],
                results['y_pred'][evening_mask], alpha=0.5)
    plt.plot([results['y_true'][evening_mask].min(),
              results['y_true'][evening_mask].max()],
             [results['y_true'][evening_mask].min(),
              results['y_true'][evening_mask].max()],
             'r--', lw=2, label='Perfect prediction')
    
    # Add trend line
    z = np.polyfit(results['y_true'][evening_mask], results['y_pred'][evening_mask], 1)
    p = np.poly1d(z)
    plt.plot([results['y_true'][evening_mask].min(),
              results['y_true'][evening_mask].max()],
             [p(results['y_true'][evening_mask].min()),
              p(results['y_true'][evening_mask].max())],
             'g--', lw=2, label='Trend line')
    plt.legend()
    plt.xlabel('Actual DO (mg/L)')
    plt.ylabel('Predicted DO (mg/L)')
    plt.title('Evening: Predicted vs actual DO levels')
    
    plt.tight_layout()
    plt.show()
    
		# Calculate SHAP values
    
    # Get the random forest model and preprocessor from pipeline
    model = best_estimator.named_steps['regressor']
    preprocess_pipeline = best_estimator[:-1]
    X_transformed = preprocess_pipeline.transform(X)
    feature_names = best_estimator.feature_names_in_
        
    # Create SHAP explainer
    shap_values = get_shap_values(model, X_transformed, attribute_names=feature_names)
    plot_shap(shap_values)
    
    return {
        'best_params': grid_search.best_params_,
        'best_score': grid_search.best_score_,
        'best_estimator': grid_search.best_estimator_,
        'results': results,
        'feature_names': feature_names
    }